# AMOCarray demo

The purpose of this notebook is to demonstrate the functionality of `AMOCarray`.

The demo is organised to show

- Step 1: Loading and plotting a sample dataset

- Step 2: Exploring the dataset attributes and variables.

Note that when you submit a pull request, you should `clear all outputs` from your python notebook for a cleaner merge.


In [ ]:
import pathlib
import sys

script_dir = pathlib.Path().parent.absolute()
parent_dir = script_dir.parents[0]
sys.path.append(str(parent_dir))

import xarray as xr
import os
import pooch
from amocarray import readers, writers, plotters, tools, utilities


In [ ]:
# Specify the path for writing datafiles
data_path = os.path.join(parent_dir, 'data')

In [ ]:
# Load data from data/moc_transports
ds = readers.load_sample_dataset()
print(ds)

In [ ]:
plotters.show_contents(ds)

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(12, 5))

# Plot stream function
ds['stream_function_mar'].plot(x='time', y='depth', cmap='viridis', ax=axes)
axes.set_title('Stream Function (MAR)')
axes.invert_yaxis()
plt.tight_layout()
plt.show()